<a href="https://colab.research.google.com/github/jorgewellison/classifiers-tcc/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Importação das bibliotecas**

In [4]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from yellowbrick.classifier import ConfusionMatrix
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier, export
from sklearn.model_selection import cross_val_score, cross_val_predict


**Importando previsores e classe (tratados no pré-processamento)**

previsores -> variáveis independentes - X

classe -> variável dependente - y

In [5]:
previsores = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/previsores.csv") 
classe = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid-idhm/classe.csv") 

**Criação do KFold com 10 iterações e Shuffle para embaralhar as linhas, desagrupando os municipios de seus estados**

In [8]:
cv = KFold(n_splits=10, random_state=0, shuffle=True)

**Criação do modelo (máquina de vetores de suporte) e treinamento**

In [25]:
svm = SVC(kernel='poly')
svm = svm.fit(previsores, classe.values.ravel())

**Testes com validação cruzada**

In [26]:
#a função cross_val_score recebe por parâmetro o modelo
#todos os dados para treino, os dados de classes, a quandidade de fodls e o metodo de avaliação
scores = cross_val_score(svm, previsores, classe.values.ravel(), cv=cv, scoring='accuracy')

**Predição e Matriz de confusão**

In [27]:
predictions = cross_val_predict(svm, previsores, classe.values.ravel(), cv=cv)
confusao = confusion_matrix(classe, predictions)
confusao

array([[3576,    8,    0],
       [1654,   13,    0],
       [ 312,    2,    0]])

**Imprimindo os resultados dos folds**

In [28]:
scores

array([0.64093357, 0.6427289 , 0.61759425, 0.64631957, 0.63554758,
       0.65107914, 0.64568345, 0.66366906, 0.66007194, 0.64568345])

**Taxa de acerto**

In [29]:
scores.mean() #porcentagem de precisão

0.6449310928277127

**Métricas de avaliação**

In [30]:
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.645 (0.012)


**Taxa de erro**

In [31]:
taxa_erro = 1 - scores.mean()
taxa_erro

0.35506890717228734